Input File Creation
===========
First let's start with some tools to create input files for a given deployment schedule.

In [1]:
import os
import sys
import uuid
import json
import time
import subprocess
from math import ceil
from copy import deepcopy

import numpy as np
import pandas as pd
import cymetric as cym
%matplotlib inline
import matplotlib.pyplot as plt
import george

import dtw

/home/scopatz/miniconda/lib/python3.5/importlib/_bootstrap.py:222: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)
/home/scopatz/miniconda/lib/python3.5/importlib/_bootstrap.py:222: QAWarning: pyne.material is not yet QA compliant.
  return f(*args, **kwds)
/home/scopatz/miniconda/lib/python3.5/importlib/_bootstrap.py:222: QAWarning: pyne.enrichment is not yet QA compliant.
  return f(*args, **kwds)


In [2]:
with open('once-through.json') as f:
    BASE_SIM = json.load(f)
DURATION = BASE_SIM['simulation']['control']['duration']
YEARS = ceil(DURATION / 12)
MONTH_SHUFFLE = (1, 7, 10, 4, 8, 6, 12, 2, 5, 9, 11, 3)
NULL_SCHEDULE = {'build_times': [{'val': 1}], 
                 'n_build': [{'val': 0}], 
                 'prototypes': [{'val': 'LWR'}]}
LWR_PROTOTYPE = {'val': 'LWR'}
OPT_H5 = 'opt.h5'

In [3]:
BASE_SIM['simulation']['region']['institution']['config']['DeployInst']

{'build_times': [{'val': 1}],
 'n_build': [{'val': 0}],
 'prototypes': [{'val': 'LWR'}]}

In [4]:
def deploy_inst_schedule(Θ):
    if np.sum(Θ) == 0: 
        return NULL_SCHEDULE
    sched = {'build_times': {'val': []},
             'n_build': {'val': []},
             'prototypes': {'val': []}}
    build_times = sched['build_times']['val']
    n_build = sched['n_build']['val']
    prototypes = sched['prototypes']['val']
    m = 0
    for i, θ in enumerate(Θ):
        if θ <= 0:
            continue
        build_times.append(i*12 + MONTH_SHUFFLE[m])
        n_build.append(int(θ))
        prototypes.append('LWR')
        m = (m + 1) % 12
    return sched

def make_sim(Θ, fname='sim.json'):
    sim = deepcopy(BASE_SIM)
    inst = sim['simulation']['region']['institution']
    inst['config']['DeployInst'] = deploy_inst_schedule(Θ)
    with open(fname, 'w') as f:
        json.dump(sim, f)
    return sim

In [5]:
s = make_sim([])

In [6]:
s['simulation']['region']['institution']['config']['DeployInst']

{'build_times': [{'val': 1}],
 'n_build': [{'val': 0}],
 'prototypes': [{'val': 'LWR'}]}

Simulate
=========
Now let's build some tools to run simulations and extract a GWe time series.

In [7]:
def run(fname='sim.json', out=OPT_H5):
    """Runs a simulation and returns the sim id."""
    cmd = ['cyclus', '--warn-limit', '0', '-o', out, fname]
    proc = subprocess.run(cmd, check=True, universal_newlines=True, stdout=subprocess.PIPE)
    simid = proc.stdout.rsplit(None, 1)[-1]
    return simid

ZERO_GWE = pd.DataFrame({'GWe': np.zeros(YEARS)}, index=np.arange(YEARS))
ZERO_GWE.index.name = 'Time'

def extract_gwe(simid, out=OPT_H5):
    """Computes the annual GWe for a simulation."""
    with cym.dbopen(out) as db:
        evaler = cym.Evaluator(db)
        raw = evaler.eval('TimeSeriesPower', conds=[('SimId', '==', uuid.UUID(simid))])
    ano = pd.DataFrame({'Time': raw.Time.apply(lambda x: x//12), 
                        'GWe': raw.Value.apply(lambda x: 1e-3*x/12)})
    gwe = ano.groupby('Time').sum()
    gwe = (gwe + ZERO_GWE).fillna(0.0)
    return np.array(gwe.GWe)

Distancing
========
Now let's build some tools to distance between a GWe time series and a demand curve.

In [8]:
DEFAULT_DEMAND = 90 * (1.01**np.arange(YEARS))  # 1% growth

In [9]:
def d(g, f=None):
    """The dynamic time warping distance between a GWe time series and a demand curve."""
    f = DEFAULT_DEMAND if f is None else f
    rtn = dtw.distance(f[:, np.newaxis], g[:, np.newaxis])
    return rtn

def αd(g, f=None, dtol=1e-5):
    """Computes the mininmal α and the DTW d."""
    f = DEFAULT_DEMAND if f is None else f
    C = dtw.cost_matrix(f[:, np.newaxis], g[:, np.newaxis])
    d = dtw.distance(cost=C)
    d_t = np.diagonal(C) / (np.arange(1, C.shape[0] + 1) + np.arange(1, C.shape[1] + 1))
    #α = np.argwhere(np.cumsum(d_t <= dtol) != np.arange(1, len(d_t) + 1))[0,0]
    filt = np.argwhere(d_t <= dtol)
    #α = filt[-1,-1] if len(filt) > 0 else 0
    α = filt[0,0] if len(filt) > 0 else 0
    print("Simulation α", α)
    print("Simulation d(t)", d_t)
    return α, d

def gwed(Θ, f=None, dtol=1e-5, find_α=False):
    """For a given deployment schedule Θ, return the GWe time series and the distance 
    to the demand function f.
    """
    make_sim(Θ)
    simid = run()
    gwe = extract_gwe(simid)
    if find_α:
        rtn = αd(gwe, f=f, dtol=dtol)
    else:
        rtn = d(gwe, f=f)
    return (gwe,) + rtn

Initialize Optimization
===============
Now let's start with a couple of simple simulations

In [10]:
N = np.asarray(np.ceil(4*(1.01)**np.arange(YEARS)), dtype=int)  # max annual deployments
Θs = [] # deployment schedules
G = []  # GWe per sim
D = []  # distances per sim
if os.path.isfile(OPT_H5):
    os.remove(OPT_H5)

In [11]:
def add_sim(Θ, f=None, dtol=1e-5):
    """Add a simulation to the known simulations by performing the simulation."""
    g_s, α, d_s = gwed(Θ, f=f, dtol=dtol, find_α=True)
    Θs.append(Θ)
    G.append(g_s)
    D.append(d_s)
    return α

First, add a schedule where nothing is deployed, leaving the initial facilities to retire.

In [12]:
add_sim(np.zeros(YEARS, dtype=int))

Simulation α 0
Simulation d(t) [  2.41666667   1.25416667   1.41538889   1.86076125   2.45404509
   3.15677962   3.86915493   4.60898025   5.39078449   6.1566231
   6.96053576   7.75174741   8.5528022    9.37385425  10.17790883
  10.99711888  11.82176125  12.64427633  13.47185668  14.30609232
  15.1383523   15.98051677  16.81703982  17.65927438  18.5067591
  19.35590067  20.20518339  21.05479825  21.9034751   22.75567063
  23.61123614  24.46743917  25.32819345  26.18960111  27.05297339
  27.92063499  28.78903031  29.66046217  30.53379372  31.40800336
  32.27607323  33.11875855  33.93798615  34.73550921  35.51292657
  36.27169955  37.01316655  37.73855589  38.44899707  39.14553063]


0

Next, add a simulation that is the max deployment schedule to bound the space

In [13]:
add_sim(N)

Simulation α 0
Simulation d(t) [  0.58333333   1.775        2.24294444   2.95590542   3.23885215
   3.68286723   3.83044314   4.26916371   4.47937619   4.57581687
   4.64269744   4.86541972   5.11518614   5.29892753   5.408577
   5.58731819   5.74206029   5.92064231   6.02890973   6.24217444
   6.4302928    6.58386303   6.69738899   6.8847273    7.09464632
   7.26987853   7.42426409   7.65957925   7.87632392   8.13065033
   8.32350559   8.58619679   8.84526525   9.05475196   9.2603816
   9.54726035   9.85589107  10.1215464   10.36060699  10.62950344
  10.89042524  11.19979886  11.50443031  11.92755569  12.35706369
  12.78468028  13.20478458  13.70506913  14.26633761  14.80240312]


0

Optimizer
=======
Now let's add some tools to do the estimation phase of the optimization.

In [14]:
Γ = 250
np.random.seed(424242)

In [15]:
def gp_gwe(Θs, G, α, T=None, tol=1e-6, verbose=False):
    """Create a Gaussian process regression model for GWe."""
    S = len(G)
    T = YEARS if T is None else T
    t = np.arange(T)
    P = len(Θs[0])
    ndim = P + 1 - α
    y_mean = np.mean(G)
    y = np.concatenate(G)
    x = np.empty((S*T, ndim), dtype=int)
    for i in range(S):
        x[i*T:(i+1)*T, 0] = t
        x[i*T:(i+1)*T, 1:] = Θs[i][np.newaxis, α:]
    yerr = tol * y_mean
    #kernel = float(y_mean) * george.kernels.ExpSquaredKernel(1.0, ndim=ndim)
    #for p in range(P):
    #    kernel *= george.kernels.ExpSquaredKernel(1.0, ndim=ndim)
    #kernel = float(y_mean) * george.kernels.Matern52Kernel(1.0, ndim=ndim)
    kernel = float(y_mean) * george.kernels.Matern32Kernel(1.0, ndim=ndim)
    gp = george.GP(kernel, mean=y_mean)
    gp.compute(x, yerr=yerr, sort=False)
    gp.optimize(x, y, yerr=yerr, sort=False, verbose=verbose)
    return gp, x, y

def predict_gwe(Θ, gp, y, α, T=None):
    """Predict GWe for a deployment schedule Θ and a GP."""
    T = YEARS if T is None else T
    t = np.arange(T)
    P = len(Θ)
    ndim = P + 1 - α
    x = np.empty((T, ndim), dtype=int)
    x[:,0] = t
    x[:,1:] = Θ[np.newaxis,α:]
    mu = gp.predict(y, x, mean_only=True)
    return mu

def gp_d_inv(θ_p, D_inv, tol=1e-6, verbose=False):
    """Computes a Gaussian process model for a deployment parameter."""
    S = len(D)
    ndim = 1
    x = θ_p
    y = D_inv
    y_mean = np.mean(y)
    yerr = tol * y_mean
    kernel = float(y_mean) * george.kernels.ExpSquaredKernel(1.0, ndim=ndim)
    gp = george.GP(kernel, mean=y_mean, solver=george.HODLRSolver)
    gp.compute(x, yerr=yerr, sort=False)
    gp.optimize(x, y, yerr=yerr, sort=False, verbose=verbose)
    return gp, x, y

def weights(Θs, D, N, Nlower, α, tol=1e-6, verbose=False):
    P = len(N)
    θ_ps = np.array(Θs)
    D = np.asarray(D)
    D_inv = D**-1 
    W = [None] * α
    for p in range(α, P):
        θ_p = θ_ps[:,p]
        gp, _, _ = gp_d_inv(θ_p, D_inv, tol=tol, verbose=verbose)
        range_p = np.arange(Nlower[p], N[p] + 1)
        d_inv_np = gp.predict(D_inv, range_p, mean_only=True)
        if np.all(np.isnan(d_inv_np)) or np.all(d_inv_np <= 0.0):
            # try D, instead of D^-1
            #gp, _, _ = gp_d_inv(θ_p, D, tol=tol, verbose=verbose)
            #d_np = gp.predict(D, np.arange(0, N[p] + 1), mean_only=True)
            # try setting the shortest d to 1, all others 0.
            #d_inp_np = np.zeros(N[p] + 1, dtype='f8')
            #p_min = np.argmin(D)
            #d_inv_np[np.argwhere(θ_p[p_min] == range_p)] = 1.0
            # try Poisson dist centered at min.
            p_min = np.argmin(D)
            lam = θ_p[p_min]
            fact = np.cumprod([1.0] + list(range(1, N[p] + 1)))[Nlower[p]:N[p] + 1]
            d_inv_np = np.exp(-lam) * (lam**range_p) / fact
        if np.any(d_inv_np < 0.0):
            d_inv_np[d_inv_np < 0.0] = np.min(d_inv_np[d_inv_np > 0.0])
        d_inv_np_tot = d_inv_np.sum()
        w_p = d_inv_np / d_inv_np_tot
        W.append(w_p)
    return W

def guess_scheds(Θs, W, Γ, gp, y, α, T=None):
    """Guess a new deployment schedule, given a number of samples Γ, weights W, and 
    Guassian process for the GWe.
    """
    P = len(W)
    Θ_γs = np.empty((Γ, P), dtype=int)
    Θ_γs[:, :α] = Θs[0][:α]
    for p in range(α, P):
        w_p = W[p]
        Θ_γs[:, p] = np.random.choice(len(w_p), size=Γ, p=w_p)
    Δ = []
    for γ in range(Γ):
        Θ_γ = Θ_γs[γ]
        g_star = predict_gwe(Θ_γ, gp, y, α, T=T)
        d_star = d(g_star)
        Δ.append(d_star)
    γ = np.argmin(Δ)
    Θ_γ = Θ_γs[γ]
    print('hyperparameters', gp.kernel[:])
    #print('Θ_γs', Θ_γs)
    #print('Θ_γs[γ]', Θ_γs[γ])
    #print('Predition', Δ[γ], Δ)
    return Θ_γ

def estimate(Θs, G, D, N, Nlower, Γ, α, T=None, tol=1e-6, verbose=False):
    """Runs an estimation step, returning a new deployment schedule."""
    gp, x, y = gp_gwe(Θs, G, α, T=T, tol=tol, verbose=verbose)
    W = weights(Θs, D, N, Nlower, α, tol=tol, verbose=verbose)
    Θ = guess_scheds(Θs, W, Γ, gp, y, α, T=T)
    return Θ

def optimize(MAX_D=0.1, MAX_S=12, T=None, tol=1e-6, dtol=1e-5, verbose=False):
    global Θs, G, D
    α = 0
    s = 2
    z = 2
    n = N
    nlower = n0 = np.zeros(len(N), dtype=int)
    while MAX_D < D[-1] and s < MAX_S and α + 1 < YEARS:
        print(s)
        print('-'*18)
        Gprev = np.array(G[:z])
        t0 = time.time()
        Θ = estimate(Θs, G, D, n, nlower, Γ, α, T=T, tol=tol, verbose=verbose)
        t1 = time.time()
        α_s = add_sim(Θ, dtol=dtol)
        t2 = time.time()
        print('Estimate time:   {0} min {1} sec'.format((t1-t0)//60, (t1-t0)%60))
        print('Simulation time: {0} min {1} sec'.format((t2-t1)//60, (t2-t1)%60))
        print(D)
        sys.stdout.flush()
        idx = [int(i) for i in np.argsort(D)[:z]]
        if D[-1] == max(D):
            idx.append(-1)
        #elif len(G) == z + 2  and np.allclose(G[:z], Gprev):
        #    n = np.array([Θs[0] + 1, N], dtype=int).min(axis=0)
        #    nlower = np.array([Θs[0] - 1, n0], dtype=int).max(axis=0)
        #    print('New N-upper', n)
        #    print('New N-lower', nlower)
        #if (α < α_s) and ((len(D) - 1) in idx[:2]):
        #if (α < α_s) and (len(D) == idx[0] + 1):
        if (len(D) == idx[0] + 1):
            print('Update α: {0} -> {1}'.format(α, α_s))
            α = α_s
        elif α > 0:
            print('Update α: {0} -> {1}'.format(α, α - 1))
            α -= 1
        Θs = [Θs[i] for i in idx]
        G = [G[i] for i in idx]
        D = [D[i] for i in idx]
        s += 1
        print()

Simulate
=======
Ok! Let's try it.

In [16]:
%%time
optimize(MAX_S=25, dtol=1.0)

2
------------------
hyperparameters [ 8.44155389  5.41756185]
Simulation α 0
Simulation d(t) [ 0.58333333  1.775       2.21516667  2.39340542  2.40551882  2.39120056
  2.53282409  2.50353871  2.69233916  2.61748354  2.50120128  2.31826194
  2.22241377  2.22391099  2.19459305  2.10073175  2.03560491  1.84391852
  1.6843353   1.4830635   1.3407385   1.20100328  1.10784502  1.09108611
  1.08095762  1.0796402   1.13644692  1.24791211  1.27886217  1.28815058
  1.31818331  1.3368632   1.42145781  1.46680532  1.41932333  1.47723938
  1.52192561  1.55221627  1.59517525  1.67674125  1.81029937  1.92633861
  2.02109743  2.16642632  2.3627625   2.54999477  2.78594197  2.97299346
  3.26725157  3.54861946]
Estimate time:   0.0 min 6.830390691757202 sec
Simulation time: 0.0 min 7.684805393218994 sec
[39.145530632616016, 14.802403115793279, 3.5486194583707338]
Update α: 0 -> 0

3
------------------
hyperparameters [ 7.49342343  4.19292322]
Simulation α 43
Simulation d(t) [ 1.04166667  1.775       2.

In [17]:
Θs[-1]

array([3, 5, 3, 0, 3, 4, 4, 5, 3, 1, 1, 1, 2, 2, 4, 4, 5, 3, 3, 2, 3, 2, 3,
       4, 2, 6, 4, 4, 2, 3, 3, 3, 3, 0, 5, 6, 2, 6, 5, 2, 1, 0, 7, 5, 0, 0,
       5, 6, 1, 7])

In [18]:
G[-1]

array([  87.91666667,   95.91666667,   96.83333333,   96.66666667,
         96.08333333,   96.08333333,   98.58333333,   97.41666667,
        103.        ,  100.75      ,   99.75      ,   97.58333333,
         97.25      ,   98.        ,   97.33333333,   96.91666667,
        102.25      ,  101.        ,  103.41666667,  101.58333333,
        102.91666667,  103.5       ,  101.83333333,  101.33333333,
        104.91666667,  107.75      ,  107.66666667,  108.91666667,
        109.33333333,  110.5       ,  110.41666667,  108.83333333,
        112.83333333,  110.83333333,  110.58333333,  112.16666667,
        113.75      ,  118.41666667,  121.33333333,  119.5       ,
        119.91666667,  120.75      ,  124.08333333,  128.41666667,
        132.83333333,  130.83333333,  135.33333333,  135.41666667,
        144.        ,  146.33333333])

In [19]:
DEFAULT_DEMAND

array([  90.        ,   90.9       ,   91.809     ,   92.72709   ,
         93.6543609 ,   94.59090451,   95.53681355,   96.49218169,
         97.45710351,   98.43167454,   99.41599129,  100.4101512 ,
        101.41425271,  102.42839524,  103.45267919,  104.48720598,
        105.53207804,  106.58739882,  107.65327281,  108.72980554,
        109.8171036 ,  110.91527463,  112.02442738,  113.14467165,
        114.27611837,  115.41887955,  116.57306835,  117.73879903,
        118.91618702,  120.10534889,  121.30640238,  122.5194664 ,
        123.74466107,  124.98210768,  126.23192876,  127.49424804,
        128.76919052,  130.05688243,  131.35745125,  132.67102577,
        133.99773602,  135.33771338,  136.69109052,  138.05800142,
        139.43858144,  140.83296725,  142.24129692,  143.66370989,
        145.10034699,  146.55135046])

In [20]:
D**-1

TypeError: unsupported operand type(s) for ** or pow(): 'list' and 'int'

In [ ]:
1.7 / 40